Formal data for R-esque style.

In [28]:
# Training Data
data_y_vec = np.array(data_y)
data_x_vec = np.array(data_x).reshape(-1,)

# Testing Data
data_y_test_vec = np.array(data_y_test)
data_x_test_vec = np.array(data_x_test).reshape(-1,)

#### Regression Benchmark Models
1. LOWESS
2. Smothing Splines
3. Deep feed-forward Neural Networks

### Imports

In [18]:
import numpy as np
import matplotlib.pyplot as plt

## LOWESS

#### Imports for Lowess

In [19]:
import statsmodels.api as sm

Perform Lowess Regression

In [20]:
# LOESS = sm.nonparametric.lowess
# f_hat_LOESS = LOESS(data_y_vec,data_x_vec.reshape(-1,))
# LOESS_x = list(zip(*f_hat_LOESS))[0]
# f_hat_LOESS = list(zip(*f_hat_LOESS))[1]

In [ ]:
def get_LOESS(data_x,data_x_test,data_y):
    # Initializations #
    #-----------------#
    from scipy.interpolate import interp1d
    import statsmodels.api as sm
    
    # Coerce Data #
    #=============#
    # Training Data
    data_y_vec = np.array(data_y)
    data_x_vec = np.array(data_x).reshape(-1,)
    # Testing Data
    data_x_test_vec = np.array(data_x_test).reshape(-1,)
    
    # Train LOESS #
    #=============#
    LOESS = sm.nonparametric.lowess
    f_hat_LOESS = LOESS(data_y_vec,data_x_vec.reshape(-1,))
    LOESS_x = list(zip(*f_hat_LOESS))[0]
    f_hat_LOESS = list(zip(*f_hat_LOESS))[1]
    
    # Get LOESS Prediction(s) #
    #-------------------------#
    # Train
    f = interp1d(LOESS_x, f_hat_LOESS, bounds_error=False)
    LOESS_prediction_train = f(data_x_vec)
    LOESS_prediction_train = np.nan_to_num(LOESS_prediction_train)
    # Test
    LOESS_prediction_test = f(data_x_test_vec)
    LOESS_prediction_test = np.nan_to_num(LOESS_prediction_test)
    
    # Return LOESS Outputs
    return LOESS_prediction_train, LOESS_prediction_test

In [ ]:
LOESS_prediction_train, LOESS_prediction_test = get_LOESS(data_x,data_x_test,data_y)

## Smoothing Splines

#### Imports for Smoothing Splines
This next module calls R functions directly, since smoothing splines is well-developed there *(at the moment)* but not so much so in Python.

In [21]:
import rpy2.robjects as robjects

Perform Smoothing Splines Regression

In [22]:
import rpy2.robjects as robjects # Work directly from R (since smoothing splines packages is better)
r_y = robjects.FloatVector(data_y_vec)
r_x = robjects.FloatVector(data_x_vec)

r_smooth_spline = robjects.r['smooth.spline'] #extract R function# run smoothing function
spline1 = r_smooth_spline(x=r_x, y=r_y, spar=0.7)
f_hat_smoothing_splines=np.array(robjects.r['predict'](spline1,robjects.FloatVector(data_x_vec)).rx2('y'))

# Kernel Regression

### Generate: Train-Set Predictions

In [29]:
# Splines
f_hat_smoothing_splines_train=np.array(robjects.r['predict'](spline1,robjects.FloatVector(data_x_vec)).rx2('y'))

# Feed-forward Neural Network
# ffNN_prediction_test = ffNN.predict(data_x_test)

# LOESS
# from scipy.interpolate import interp1d
# f = interp1d(LOESS_x, f_hat_LOESS, bounds_error=False)
# LOESS_prediction_train = f(data_x_vec)
# LOESS_prediction_train = np.nan_to_num(LOESS_prediction_train)

### Generate: Test-Set Predictions

In [29]:
# Splines
f_hat_smoothing_splines_test=np.array(robjects.r['predict'](spline1,robjects.FloatVector(data_x_test_vec)).rx2('y'))

# Feed-forward Neural Network
# ffNN_prediction_test = ffNN.predict(data_x_test)

# LOESS
# from scipy.interpolate import interp1d
# f = interp1d(LOESS_x, f_hat_LOESS, bounds_error=False)
# LOESS_prediction_test = f(data_x_test_vec)
# LOESS_prediction_test = np.nan_to_num(LOESS_prediction_test)

# Feed-Forward Neural Network

In [ ]:
ffNN_y_hat_train,ffNN_y_hat_test = build_ffNN(n_folds = CV_folds, 
                                             n_jobs = n_jobs, 
                                             n_iter = n_iter, 
                                             param_grid_in = param_grid_Vanilla_Nets, 
                                             X_train = data_x, 
                                             y_train = data_y,
                                             X_test = data_x_test)